# Agentes Inteligentes #

Este notebook utiliza implementaciones del módulo agent.py. Comencemos importando todo desde el módulo de agentes.

In [1]:
# pip install qpsolvers
# Recomendación: Instalar qpsolvers desde la consola en el caso que la siguiente celda no pueda ejecutarse

In [1]:
from agents import *
from notebook import psource

## CONTENIDOS

* Introducción
* Agente
* Entorno
* Agente Simple y Entorno
* Agentes en un Entorno 2-D
* Entorno Wumpus

## INTRODUCCIÓN

Un agente, es cualquier cosa que pueda percibir su <b>entorno</b> a través de sensores y actuar sobre ese entorno a través de actuadores basados en su <b>programa agente</b>. Puede ser un perro, un robot o incluso usted. Siempre que puedas percibir el entorno y actuar sobre él, eres un agente. Este cuaderno explicará cómo implementar un agente simple, crear un entorno e implementar un programa que ayude al agente a actuar sobre el entorno en función de sus percepciones.

## AGENTE

Veamos ahora cómo definimos un agente. Ejecute la siguiente celda para ver cómo se define `Agente` en el módulo de agents.

In [17]:
psource(Agent)

`Agent` tiene dos métodos.
* `__init__(self, program=None)`: El constructors define varios atributos de Agent. Estos incluyen

    * `alive`: Que realiza un seguimiento de si el agente está vivo o no 
    
    * `bump`: que rastrea si el agente choca con un borde del entorno (for eg, a wall in a park)
    
    * `holding`: que es una lista que contiene las "Cosas"(Things) que tiene un agente 
    
    * `performance`: que evalúa las métricas de desempeño del agente 
    
    * `program`: que es el programa del agente y asigna las percepciones de un agente a las acciones en el entorno. Si no se proporciona una implementación, el valor predeterminado es pedir al usuario que proporcione acciones para cada percepción.
    
* `can_grab(self, thing)`: Se utiliza cuando un entorno contiene cosas que un agente puede agarrar y transportar. De forma predeterminada, un agente no puede llevar nada..

## ENTORNO
Ahora, veamos cómo se definen los entornos. Al ejecutar la siguiente celda, se mostrará una implementación de la clase abstracta `Environment`

In [18]:
psource(Environment)

`Environment` ¡la clase tiene muchos métodos! Pero la mayoría de ellos son increíblemente simples, así que veamos los que usaremos en este cuaderno.

* `thing_classes(self)`: Devuelve una matriz estática de subclases `Thing` que determinan qué cosas están permitidas en el entorno y cuáles no

* `add_thing(self, thing, location=None)`: Agrega algo al entorno en la ubicación.

* `run(self, steps)`: Ejecuta un entorno con el agente en él durante un número determinado de pasos.

* `is_done(self)`: Devuelve verdadero si se ha cumplido el objetivo del agente y el entorno

Las siguientes dos funciones deben ser implementadas por cada subclase de `Environment` para que el agente reciba percepciones y ejecute acciones. 

* `percept(self, agent)`: Dado un agente, este método devuelve una lista de percepciones que el agente ve en el momento actual.

* `execute_action(self, agent, action)`: El entorno reacciona a una acción realizada por un agente determinado. Los cambios pueden hacer que el agente experimente nuevas percepciones u otros elementos que reaccionen a la entrada del agente.

## AGENTE SIMPLE Y ENTORNO

Comencemos por usar la clase `Agent` para crear nuestro primer agente: un perro ciego (blind dog).

In [19]:
class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

Can't find a valid program for BlindDog, falling back to default.


Lo que acabamos de hacer es crear un perro que solo puede sentir lo que hay en su ubicación (ya que es ciego) y puede comer o beber. Veamos si está vivo ...

In [20]:
print(dog.alive)

True


![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
Este es nuestro perro. ¿Qué tan genial es él? Bueno, tiene hambre y necesita ir a buscar comida. Para que él haga esto, necesitamos darle un programa. Pero antes de eso, creemos un parque para que juegue nuestro perro.

### ENTORNO - Parque

Un parque es un ejemplo de entorno porque nuestro perro puede percibirlo y actuar sobre él. La clase `Environment` es una clase abstracta, por lo que tendremos que crear nuestra propia subclase a partir de ella antes de poder usarla.

In [21]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


### PROGRAMA - BlindDog
Ahora que tenemos una clase <b> Park </b>, re-implementamos nuestro <b> BlindDog </b> para poder bajar y comer o beber agua solo si está presente.


In [22]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Ahora es el momento de implementar un módulo de <b>programa</b> para nuestro perro. Un programa controla cómo actúa el perro en su entorno. Nuestro programa será muy simple y se muestra en la siguiente tabla.

<table>
     <tr>
         <td> <b> Percepción: </b> </td>
         <td> Siente la comida </td>
         <td> Siente el agua </td>
         <td> No sienta nada </td>
    </tr>
    <tr>
        <td> <b> Acción: </b> </td>
        <td> comer </td>
        <td> beber </td>
        <td> moverse hacia abajo </td>
    </tr>
        
</table>

In [23]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

Ahora ejecutemos nuestra simulación creando un parque con algo de comida, agua y nuestro perro.

In [24]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(5)

BlindDog decided to move down at location: 1
BlindDog decided to move down at location: 2
BlindDog decided to move down at location: 3
BlindDog decided to move down at location: 4
BlindDog ate Food at location: 5


Observe que el perro se movió de la ubicación 1 a la 4, en 4 pasos y comió comida en la ubicación 5 en el quinto paso.

Continuemos esta simulación por 5 pasos más.

In [14]:
park.run(5)

BlindDog decided to move down at location: 5
BlindDog decided to move down at location: 6
BlindDog drank Water at location: 7


¡Perfecto! Observe cómo la simulación se detuvo después de que el perro bebió el agua: agotar toda la comida y el agua finaliza nuestra simulación, como lo habíamos definido antes. Agreguemos un poco más de agua y veamos si nuestro perro puede alcanzarla.

In [15]:
park.add_thing(water, 15)
park.run(10)

BlindDog decided to move down at location: 7
BlindDog decided to move down at location: 8
BlindDog decided to move down at location: 9
BlindDog decided to move down at location: 10
BlindDog decided to move down at location: 11
BlindDog decided to move down at location: 12
BlindDog decided to move down at location: 13
BlindDog decided to move down at location: 14
BlindDog drank Water at location: 15


Arriba, aprendimos a implementar un agente, su programa y un entorno en el que actúa. Sin embargo, este fue un caso muy simple. ¡Intentemos agregarle complejidad creando un entorno bidimensional!


## AGENTES EN UN ENTORNO 2D

Para que no leamos tantos registros de lo que hizo nuestro perro, agregamos un poco de gráficos mientras hacemos nuestro Park 2D. Para hacerlo, necesitaremos convertirlo en una subclase de <b> GraphicEnvironment </b> en lugar de Environment. Los parques implementados subclasificando la clase <b> GraphicEnvironment </b> le agregan estas propiedades adicionales:

 - Nuestro parque está indexado en el 4º cuadrante del plano X-Y.
 - Cada vez que creamos una subclase de parque con <b> GraphicEnvironment </b>, necesitamos definir los colores de todas las cosas que planeamos poner en el parque. Los colores se definen en el típico [<b> formato digital RGB de 8 bits </b>] (https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), común en la web.
 - Las cercas se agregan automáticamente a todos los parques para que nuestro perro no salga de los límites del parque, ¡simplemente no es seguro para los perros ciegos estar solos fuera del parque! <b> GraphicEnvironment </b> proporciona la función `is_inbounds` para comprobar si nuestro perro intenta salir del parque.
 
Primero intentemos actualizar nuestro entorno unidimensional `Park` simplemente reemplazando su superclase por` GraphicEnvironment`. 

In [16]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Ahora probemos este nuevo parque con nuestro mismo perro, comida y agua. Coloreamos a nuestro perro con un bonito rojo y marcamos la comida y el agua con naranja y azul respectivamente.

In [17]:
# park width is set to 5, and height to 20
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) 

dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("BlindDog starts at (1,1) facing downwards, lets see if he can find any food!")
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


¡Agregar algunos gráficos fue una buena idea! Inmediatamente vemos que el código funciona, pero nuestro perro ciego no hace ningún uso del espacio bidimensional disponible para él. Hagamos que nuestro perro tenga más energía para que gire y se mueva hacia adelante, en lugar de bajar siempre. Al hacerlo, también necesitaremos realizar algunos cambios en nuestro entorno para poder manejar este movimiento adicional.

### PROGRAMA - EnergeticBlindDog

Hagamos que nuestro perro gire o se mueva hacia adelante al azar, excepto cuando esté en el borde de nuestro parque, en cuyo caso le hacemos cambiar su dirección explícitamente girando para evitar tratar de salir del parque. Sin embargo, nuestro perro es ciego, por lo que no sabría en qué dirección girar, solo tendría que intentarlo arbitrariamente.

<table>
    <tr>
        <td> <b> Percepción: </b> </td>
        <td> Siente la comida </td>
        <td> Siente el agua </td>
        <td> No sienta nada </td>
   </tr>
   <tr>
       <td> <b> Acción: </b> </td>
       <td> comer </td>
       <td> beber </td>
       <td>
       <tabla>
           <tr>
               <td> <b> Recuerda estar en Edge: </b> </td>
               <td> En Edge </td>
               <td> No en el borde </td>
           </tr>
           <tr>
               <td> <b> Acción: </b> </td>
               <td> Girar a la izquierda / Girar a la derecha <br> (50% - 50% de probabilidad) </td>
               <td> Girar a la izquierda / Girar a la derecha / Avanzar <br> (25% - 25% - 50% de probabilidad) </td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [21]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
        
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

### ENVIRONMENT - Park2D
También necesitamos modificar nuestro parque en consecuencia, para poder manejar todas las nuevas acciones que nuestro perro desee ejecutar. Además, tendremos que evitar que nuestro perro se mueva a lugares más allá de los límites de nuestro parque; simplemente, no es seguro para los perros ciegos estar solos fuera del parque.

In [22]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


Ahora que nuestro parque está listo para el movimiento 2D de nuestro enérgico perro, ¡probémoslo!

In [25]:
park = Park2D(10,10, color={'EnergeticBlindDog': (212,30,30), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(50)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,










## Wumpus Environment

In [27]:
from ipythonblocks import BlockGrid
from agents import *

color = {"Breeze": (225, 225, 225),
        "Pit": (0,0,0),
        "Gold": (253, 208, 23),
        "Glitter": (253, 208, 23),
        "Wumpus": (43, 27, 23),
        "Stench": (128, 128, 128),
        "Explorer": (0, 0, 255),
        "Wall": (44, 53, 57)
        }

def program(percepts):
    '''Returns an action based on it's percepts'''
    print(percepts)
    return input()

w = WumpusEnvironment(program, 7, 7)         
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    global grid, w
    draw_grid(w.get_world())
    grid.show()
    w.step()

In [30]:
step()

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


[[<Bump>], [<Breeze>], [<Bump>], [<Breeze>], [None]]
None
